# Báo cáo đồ án môn "Khoa học dữ liệu ứng dụng"

Nhóm 2:
1. [MSSV] - [Họ tên] - [Nick Github]
2. [MSSV] - [Họ tên] - [Nick Github]
3. [MSSV] - [Họ tên] - [Nick Github]
4. [MSSV] - [Họ tên] - [Nick Github]

Link thùng chứa Github của nhóm: [link]

## Mô tả bài toán

Bảo hiểm là một thứ bắt buộc khi mua một chiếc xe ô tô. Thông thường, mức chi phí đóng bảo hiểm sẽ bằng nhau. Nhưng có những người chạy xe cẩn thận, cũng có người chạy xe hay va chạm khiến cho chi phí đóng bảo hiểm cho người người như vậy không công bằng. Như vậy, cần có một mức giá khác nhau cho những người lái xe khác nhau để công bằng với khách hàng cũng như công ty bảo hiểm.

Porto Seguro - Một công ty bảo hiểm lớn ở Brazil yêu cầu một mô hình học máy tốt để dự đoán **xác suất yêu cầu bồi thường** của chủ xe nhằm đưa ra mức giá bảo hiểm hợp lý hơn.

Input/Output:
* Input: Dữ liệu lịch sử được công ty thu thập từ chủ ô tô (chủ hợp đồng bảo hiểm) từ các năm trước.
* Output: Xác suất chủ xe đó yêu cầu bồi thường bảo hiểm. Giá trị từ 0 - 1; càng về 0 có nghĩa khả năng cao chủ ô tô đó sẽ không yêu cầu bồi thường cho năm sau; càng về 1 nghĩa là chủ xe khả năng cao năm sau đòi bồi thường bảo hiểm.

Link bài toán: [Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/overview)

## Giải quyết bài toán

## Nhìn lại quá trình làm đồ án

## Tài liệu tham khảo